In [1]:
# depedencies
import getpass
from topolib.subsetDat import subsetBBox;
from topolib import IceSat2Data
import glob
import rasterio
from topolib import gda_lib

/srv/conda/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/srv/conda/lib/python3.6/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
# inputs
# if want to explcitly define bounding box, raster = None
datafolder = '/home/jovyan/data/'
datafolderOut = '/home/jovyan/data/out/'

raster = datafolder+'Tuolumne_snow-off_ASO.tif'
rasterBnd = 1

if rasterBnd == None:
    # Area of interest
    boundaries = {
        'LowerLeft_Lon': '-108.3',
        'LowerLeft_Lat': '38.8',
        'UpperRight_Lon': '-107.5',
        'UpperRight_Lat': '39.3',
    }

# Specify the variables of interest
LIce_var = ['atl06_quality_summary','delta_time','h_li','hli_sigma',\
           'latitude','longitude','segment_id','sigma_geo_h']
variables = {
    'beams': [
        '/land_ice_segments/'+LIce_var[0],
        '/land_ice_segments/'+LIce_var[1],
        '/land_ice_segments/'+LIce_var[2],
        '/land_ice_segments/'+LIce_var[3],
        '/land_ice_segments/'+LIce_var[4],
        '/land_ice_segments/'+LIce_var[5],
        '/land_ice_segments/'+LIce_var[6],
        '/land_ice_segments/'+LIce_var[7],
        '/ancillary_data/atlas_sdp_gps_epoch'
    ],
    'other': [
        '/orbit_info/cycle_number',
        '/orbit_info/rgt',
        '/orbit_info/orbit_number',
    ]
}

# Custom time range of interest
# define full_per = 1
full_per = 0
time_range = {
        'start_date': '2018-10-01',
        'end_date': '2019-01-01'
}

# Enter your Earthdata Login user name
user_id = 'jpflug'
# Enter your email address associated with your Earthdata Login account
email = 'jpflug@uw.edu'

In [16]:
# preprocessing
if rasterBnd != None:
    [L,B,R,T] = subsetBBox(raster,'epsg:32611','epsg:4326')
    # Area of interest
    boundaries = {
        'LowerLeft_Lon': L,
        'LowerLeft_Lat': B,
        'UpperRight_Lon': R,
        'UpperRight_Lat': T,
    }

In [4]:
# Earthdata Login credentials
password = getpass.getpass('Earthdata Login password: ')

# Setup a search session
search = IceSat2Data(user_id, password, variables=variables)

Earthdata Login password:  ········


In [5]:
# Show available variables
search.show_variables()

# Show available formats
search.show_formats()

['/ancillary_data',
 '/ancillary_data/atlas_sdp_gps_epoch',
 '/ancillary_data/control',
 '/ancillary_data/data_end_utc',
 '/ancillary_data/data_start_utc',
 '/ancillary_data/end_cycle',
 '/ancillary_data/end_delta_time',
 '/ancillary_data/end_geoseg',
 '/ancillary_data/end_gpssow',
 '/ancillary_data/end_gpsweek',
 '/ancillary_data/end_orbit',
 '/ancillary_data/end_region',
 '/ancillary_data/end_rgt',
 '/ancillary_data/granule_end_utc',
 '/ancillary_data/granule_start_utc',
 '/ancillary_data/qa_at_interval',
 '/ancillary_data/release',
 '/ancillary_data/start_cycle',
 '/ancillary_data/start_delta_time',
 '/ancillary_data/start_geoseg',
 '/ancillary_data/start_gpssow',
 '/ancillary_data/start_gpsweek',
 '/ancillary_data/start_orbit',
 '/ancillary_data/start_region',
 '/ancillary_data/start_rgt',
 '/ancillary_data/version',
 '/ancillary_data/land_ice',
 '/ancillary_data/land_ice/dt_hist',
 '/ancillary_data/land_ice/fit_maxiter',
 '/ancillary_data/land_ice/fpb_maxiter',
 '/ancillary_data/l

In [18]:
L,R,B,T

(-119.78855601487822, -119.2000171661589, 37.72826260439459, 38.19435356884997)

In [17]:
# Search for granules in bounding box without download through all datasets
search.search_granules(bounding_box=boundaries)

# Search for granules in bounding box without download for specific time range
search.search_granules(bounding_box=boundaries, time_range=time_range)

Number of granules:
    8
Average size of granules in MB:
    22.242888808249997
Total size in MB:
    177.943110466
Number of granules:
    6
Average size of granules in MB:
    20.415529727933333
Total size in MB:
    122.4931783676


6

In [19]:
# NOTE: file paths have to be explicitly defined 
# BEFORE submitting 

params = {}
if full_per:
    params['time_range'] = time_range

# Submit a download order to NSIDC with no time restriction
search.order_data(
    email,
    datafolderOut,
    boundaries,
    **params
)

Number of granules:
    8
Average size of granules in MB:
    22.242888808249997
Total size in MB:
    177.943110466
Order:  1
Request HTTP response:  201
order ID:  5000000320375
Data request  1  is submitting...
Initial request status is  processing
Status is not complete. Trying again.
Retry request status is:  complete_with_errors
error messages:
['161151127:NoMatchingData - No data found that matched subset constraints. '
 'Exit code 3.',
 '161176080:NoMatchingData - No data found that matched subset constraints. '
 'Exit code 3.',
 '161209518:NoMatchingData - No data found that matched subset constraints. '
 'Exit code 3.',
 '161210744:NoMatchingData - No data found that matched subset constraints. '
 'Exit code 3.',
 'PT2.558S',
 'ICESAT2']
Beginning download of zipped output...
Data request 1 is complete.


In [20]:
# latest version will move out%%
ATL06_list = glob.glob(datafolderOut+'**/*.h5')

In [21]:
import numpy as np
# read in raster data
dem_fn = raster
ds = rasterio.open(dem_fn)
dem = ds.read(1)
dem = np.ma.masked_equal(dem,gda_lib.get_ndv(ds))
## use from above??
xmin,ymin,xmax,ymax = ds.bounds

In [22]:
# read hdf5 into a geodataframe
dataset_dict={'land_ice_segments':LIce_var, 'land_ice_segments/ground_track':['x_atc']}

In [23]:
## add recursively
ATL06_gdf_list = [gda_lib.ATL06_2_gdf(x,dataset_dict) for x in ATL06_list]
ATL06_gdf_list = [x.to_crs(ds.crs) for x in ATL06_gdf_list]

In [24]:
%matplotlib notebook
import matplotlib.pyplot as plt
fig,ax = plt.subplots()
im = ax.imshow(dem,cmap='inferno',extent=[xmin,xmax,ymin,ymax])
plt.colorbar(im,label='HAE (m WGS84)')
for gdf in ATL06_gdf_list:
    gdf.plot(ax=ax)
#ATL06_gdf.plot(ax=ax)

FigureCanvasNbAgg()

In [ ]:
## add filtering

In [25]:
pwd

'/home/jovyan/topohack/notebooks'